In [ ]:
# P_0, P_1, ...
# q_Ai, q_Ao, q_Bi, q_Bo,..., q_Di, q_Do
# pow(q_x, n_x)
# exp(F*V_m/(R*T)), exp(n_x*F*V_m/(R*T))

formula='(P_0*q_Ao*q_Co*pow(q_Ai, n_Ai)*exp(n_Bi*F*V_m/(R*T)))/(P_1*q_Ai*q_Ci*pow(q_Di, n_Di))'

def templateEQdemo(formula):
    numerator,denominator = formula.split(')/(')
    numerator = numerator+')'
    denominator = '('+denominator
    
    replacements = {'*': ' '}
    
    for x in ['Ai','Ao','Bi','Bo','Ci','Co','Di','Do']:
        replacements['q_'+x] = 'q_{'+x+'}'
        replacements['n_'+x] = 'n_{'+x+'}'
        replacements['pow('+'q_{'+x+'}, n_{'+x+'})'] = 'q_{'+x+'}^'+'n_{'+x+'}'
        replacements['exp('] = 'e^{('
        replacements['F*V_m/(R*T))'] = 'F V_m/(R T))}'
        
    for i in range(-100,1):
        replacements['P_'+str(-i)] = 'p_{'+str(-i)+'}'
        
        
    for key, value in replacements.items():
        numerator = numerator.replace(key, value)
        denominator = denominator.replace(key, value)

    
    return r'$V_{SS} = \frac{'+numerator+'}{'+denominator+'}$'


In [ ]:
def _dump_issues(source_method_name, logger):
    if logger.issueCount() > 0:
        print('The method "{}" found {} issues:'.format(source_method_name, logger.issueCount()))
        for i in range(0, logger.issueCount()):
            print('    - {}'.format(logger.issue(i).description()))

def parse_model(filename, strict_mode):
    cellml_file = open(filename)
    parser = Parser(strict_mode)
    model = parser.parseModel(cellml_file.read())
    # _dump_issues("parse_model", parser)
    return model

def print_model(model):
    printer = Printer()
    s = printer.printModel(model)
    return s

def validate_model(model):
    validator = Validator()
    validator.validateModel(model)
    # _dump_issues("validate_model", validator)
    return validator.issueCount()


def flatten_model(model, importer):
    flat_model = importer.flattenModel(model)
    return flat_model

def analyse_model(model):
    analyser = Analyser()
    a = analyser.analyseModel(model)
    # _dump_issues("analyse_model", analyser)
    return a

def resolve_imports(model, base_dir, strict_mode):
    importer = Importer(strict_mode)
    importer.resolveImports(model, base_dir)
    # _dump_issues("resolve_imports", importer)
    # if model.hasUnresolvedImports():
    #     print("unresolved imports?")
    # else:
    #     print("no unresolved imports.")
    return importer

In [ ]:
def pValsFunc(singleSelection,floatSSget,dataset=False):
    
    file = open('./temporary files/speciesStoi.json')
    speciesStoi = json.load(file)
    

    # This section works for 'Template 3'
    if singleSelection.value == 'Template 3':
        # getting the stoichiometries
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]

        solve = tuple()
        if dataset==False:
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ao$'+'  ($u$mol)'][0],)
        else:
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ao$'+'  ($u$mol)'][0],)

        bounds=[[],[]]
        for i in range(6):
            bounds[0].append(0)
            bounds[1].append(np.inf)

        def func(X, p1,p2,p3,p4,p5,p6):
            ai,ao = X
            Ai,Ao = ai**n_Ai, ao**n_Ao
            V_SS = (p1* Ao - p2* Ai)/ (p3 + p4*Ao + p5*Ai*Ao + p6*Ai)
            return V_SS

        if dataset==False:
            popt, pcov = curve_fit(func,solve,[el.value for el in floatSSget if el.description=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)
        else:
            popt, pcov = curve_fit(func,solve,[floatSSget[el] for el in floatSSget.columns if el=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)


        pVals={}
        pVals[singleSelection.value]={}
        for i,el in enumerate(popt):
            pVals[singleSelection.value]['p'+str(i+1)]=el

        json_object = json.dumps(pVals, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
        with open("./temporary files/pVals.json", "w") as outfile:
            outfile.write(json_object)
            
    # This section works for 'Template 4'
    if singleSelection.value == 'Template 4':
        # getting the stoichiometries
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        
        solve = tuple()
        if dataset==False:
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bo$'+'  ($u$mol)'][0],)
        else:
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bo$'+'  ($u$mol)'][0],)
        
        bounds=[[],[]]
        for i in range(10):
            bounds[0].append(0)
            bounds[1].append(np.inf)

        def func(X, p1,p2,p3,p4,p5,p6,p7,p8,p9,p10):
            ai,ao,bi,bo = X
            Ai,Ao,Bi,Bo = ai**n_Ai, ao**n_Ao, bi**n_Bi, bo**n_Bo
            V_SS = (p1*Ao*Bi - p2*Ai*Bo)/ (p3*Ao + p4*Ai + p5*Bo + p6*Bi + p7*Ao*Bi + p8*Ai*Ao + p9*Ai*Bo + p10*Bo*Bi)
            return V_SS
        
        if dataset==False:
            popt, pcov = curve_fit(func,solve,[el.value for el in floatSSget if el.description=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)
        else:
            popt, pcov = curve_fit(func,solve,[floatSSget[el] for el in floatSSget.columns if el=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)
       

        pVals={}
        pVals[singleSelection.value]={}
        for i,el in enumerate(popt):
            pVals[singleSelection.value]['p'+str(i+1)]=el

        json_object = json.dumps(pVals, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
        with open("./temporary files/pVals.json", "w") as outfile:
            outfile.write(json_object)
            
            
            
    # This section works for 'Template 7'
    if singleSelection.value == 'Template 7':
        # getting the stoichiometries
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        n_Ci = speciesStoi[singleSelection.value]['q_Ci'][1]
        n_Co = speciesStoi[singleSelection.value]['q_Co'][1]
        
        solve = tuple()
        if dataset==False:
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ci$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Co$'+'  ($u$mol)'][0],)
        else:
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ci$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Co$'+'  ($u$mol)'][0],)
        
        bounds=[[],[]]
        for i in range(13):
            bounds[0].append(0)
            bounds[1].append(np.inf)

        def func(X, p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13):
            ai,ao,bi,bo,ci,co = X
            Ai,Ao,Bi,Bo,Ci,Co = ai**n_Ai,ao**n_Ao,bi**n_Bi,bo**n_Bo,ci**n_Ci,co**n_Co
            V_SS = (p1*Ai*Bo*Co - p2*Ao*Bi*Ci)/(p3*Ai + p4*Ao + p5*Bi + p6*Bo + p7*Ci + p8*Co + p9*Ai*Ao + p10*Bi*Bo + p11*Ci*Co + p12*Ai*Bo*Co + p13*Bi*Ci*Ao )
            return V_SS
        
        if dataset==False:
            popt, pcov = curve_fit(func,solve,[el.value for el in floatSSget if el.description=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)
        else:
            popt, pcov = curve_fit(func,solve,[floatSSget[el] for el in floatSSget.columns if el=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)

        pVals={}
        pVals[singleSelection.value]={}
        for i,el in enumerate(popt):
            pVals[singleSelection.value]['p'+str(i+1)]=el

        json_object = json.dumps(pVals, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
        with open("./temporary files/pVals.json", "w") as outfile:
            outfile.write(json_object)
            
            
            
    # This section works for 'Template 11'
    if singleSelection.value == 'Template 11':
        # getting the stoichiometries
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        n_Ci = speciesStoi[singleSelection.value]['q_Ci'][1]
        n_Co = speciesStoi[singleSelection.value]['q_Co'][1]
        n_Di = speciesStoi[singleSelection.value]['q_Di'][1]
        n_Do = speciesStoi[singleSelection.value]['q_Do'][1]
        
        solve = tuple()
        if dataset==False:
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ci$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Co$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Di$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Do$'+'  ($u$mol)'][0],)
        else:
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ci$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Co$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Di$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Do$'+'  ($u$mol)'][0],)
        
        bounds=[[],[]]
        for i in range(16):
            bounds[0].append(0)
            bounds[1].append(np.inf)

        def func(X, p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16):
            ai,ao,bi,bo,ci,co,di,do = X
            Ai,Ao,Bi,Bo,Ci,Co,Di,Do = ai**n_Ai, ao**n_Ao, bi**n_Bi, bo**n_Bo, ci**n_Ci, co**n_Co, di**n_Di, do**n_Do
            V_SS = (p1*Ai*Bo*Co*Do - p2*Ao*Bi*Ci*Di)/(p3*Ai + p4*Ao + p5*Bi + p6*Bo + p7*Ci + p8*Co + p9*Di + p10*Do + p11*Ai*Ao + p12*Bi*Bo + p13*Ci*Co + p14*Di*Do + p15*Ai*Bo*Co*Do + p16*Bi*Ci*Ao*Di )
            return V_SS
        
        if dataset==False:
            popt, pcov = curve_fit(func,solve,[el.value for el in floatSSget if el.description=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)
        else:
            popt, pcov = curve_fit(func,solve,[floatSSget[el] for el in floatSSget.columns if el=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)

        pVals={}
        pVals[singleSelection.value]={}
        for i,el in enumerate(popt):
            pVals[singleSelection.value]['p'+str(i+1)]=el

        json_object = json.dumps(pVals, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
        with open("./temporary files/pVals.json", "w") as outfile:
            outfile.write(json_object)
            

            
    # This section works for 'Template 5'
    if singleSelection.value == 'Template 5':
        # getting the stoichiometries
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]

        solve = tuple()
        if dataset==False:
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='V_m'+'  (mV)'][0],)
        else:
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='V_m'+'  (mV)'][0],)

        bounds=[[],[]]
        for i in range(12):
            bounds[0].append(0)
            bounds[1].append(np.inf)

        def func(X, p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12):
            ai,ao,bi,bo,V_m = X
            Ai,Ao,Bi,Bo = ai**n_Ai, ao**n_Ao, bi**n_Bi, bo**n_Bo
            F=96485
            R=8.31
            T=310
            if dataset==False:
                EXP = exp(F*V_m/(R*T))
            else:
                EXP = [exp(F*vm/(R*T)) for vm in V_m]

            V_SS =  (p1*Bo*Ao*EXP-p2*Ai*Bi)/(p11*Bi*Ai*Ao*EXP+p12*Bi*Bo*Ai*Ao*EXP+p3*Bi*Bo+p4*Bo*Ai+p5*Bo*Ao*EXP+p6*Bi*Ai+p7*Bi*Ao*EXP+p8*Bi*Bo*Ai+p9*Bi*Bo*Ao*EXP+p10*Bo*Ai*Ao*EXP);
            return V_SS

        if dataset==False:
            popt, pcov = curve_fit(func,solve,[el.value for el in floatSSget if el.description=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)
        else:
            popt, pcov = curve_fit(func,solve,[floatSSget[el] for el in floatSSget.columns if el=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)


        pVals={}
        pVals[singleSelection.value]={}
        for i,el in enumerate(popt):
            pVals[singleSelection.value]['p'+str(i+1)]=el

        json_object = json.dumps(pVals, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
        with open("./temporary files/pVals.json", "w") as outfile:
            outfile.write(json_object)
            
            
    # This section works for 'Template 6'
    if singleSelection.value == 'Template 6':
        # getting the stoichiometries
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        

        solve = tuple()
        if dataset==False:
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='V_m'+'  (mV)'][0],)
        else:
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='V_m'+'  (mV)'][0],)

        bounds=[[],[]]
        for i in range(21):
            bounds[0].append(0)
            bounds[1].append(np.inf)

        def func(X, p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21):
            ai,ao,bi,bo,V_m = X
            Ai,Ao,Bi,Bo = ai**n_Ai, ao**n_Ao, bi**n_Bi, bo**n_Bo
            F=96485
            R=8.31
            T=310
            if dataset==False:
                EXP = exp(F*V_m/(R*T))
                EXP2 = exp(n_Bo*F*V_m/(R*T))
            else:
                EXP = [exp(F*vm/(R*T)) for vm in V_m]
                EXP2 = [exp(n_Bo*F*vm/(R*T)) for vm in V_m]
            
            V_SS = (Bo*Bi*(p1*Ao*EXP2-p2*Ai))/(p3*Bi*Bo + p4*Bi*Bo*EXP + p5*Bi*Bo*EXP2+ p6*Bo*Ai + p7*Bo*Ai*EXP + p8*Bo*Ai*EXP2 + p9*Bo*Ao*EXP2+ p10*Bi*Ai + p11*Bi*Ao+ p12*Bi*Ao*EXP + p13*Bi*Bo*Ai + p14*Bi*Bo*Ai*EXP+ p15*Bi*Bo*Ai*EXP2 + p16*Bi*Bo*Ao*EXP2 + p17*Bi*Bo*Ao*EXP + p18*Bo*Ai*Ao*EXP2 + p19*Bi*Ai*Ao + p20*Bi*Bo*Ai*Ao*EXP2 + p21*Bi*Bo*Ai*Ao*EXP)
            return V_SS

        if dataset==False:
            popt, pcov = curve_fit(func,solve,[el.value for el in floatSSget if el.description=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)
        else:
            popt, pcov = curve_fit(func,solve,[floatSSget[el] for el in floatSSget.columns if el=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)


        pVals={}
        pVals[singleSelection.value]={}
        for i,el in enumerate(popt):
            pVals[singleSelection.value]['p'+str(i+1)]=el

        json_object = json.dumps(pVals, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
        with open("./temporary files/pVals.json", "w") as outfile:
            outfile.write(json_object)
            
    # This section works for 'Template 8'
    if singleSelection.value == 'Template 8':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]

        solve = tuple()
        if dataset==False:
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='V_m'+'  (mV)'][0],)
        else:
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='V_m'+'  (mV)'][0],)

        bounds=[[],[]]
        for i in range(11):
            bounds[0].append(0)
            bounds[1].append(np.inf)

        def func(X, p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11):
            ai,ao,bi,bo,V_m = X
            Ai,Ao,Bi,Bo = ai**n_Ai, ao**n_Ao, bi**n_Bi, bo**n_Bo
            F=96485
            R=8.31
            T=310
            if dataset==False:
                EXP = exp(F*V_m/(R*T))
                EXP2 = exp(n_Ao*F*V_m/(R*T))
            else:
                EXP = [exp(F*vm/(R*T)) for vm in V_m]
                EXP2 = [exp(n_Ao*F*vm/(R*T)) for vm in V_m]
            
            V_SS = (EXP2*(p1*Bo*Ao -p2*Bi*Ai))/(p3*Bi*EXP + p4*Bi*EXP2+ p5*Bi*Ai*EXP2+ p6*Bo*Ai*EXP2+ p7*Bi*Ao*EXP+ p8*Ai*EXP+ p9*Ai*EXP2+ p10*Bi*Ao*EXP2 + p11*Bi*Ai*Ao*EXP)
            return V_SS

        if dataset==False:
            popt, pcov = curve_fit(func,solve,[el.value for el in floatSSget if el.description=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)
        else:
            popt, pcov = curve_fit(func,solve,[floatSSget[el] for el in floatSSget.columns if el=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)


        pVals={}
        pVals[singleSelection.value]={}
        for i,el in enumerate(popt):
            pVals[singleSelection.value]['p'+str(i+1)]=el

        json_object = json.dumps(pVals, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
        with open("./temporary files/pVals.json", "w") as outfile:
            outfile.write(json_object)
           
        
    # This section works for 'Template 1'
    if singleSelection.value == 'Template 1':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        n_Ci = speciesStoi[singleSelection.value]['q_Ci'][1]
        n_Co = speciesStoi[singleSelection.value]['q_Co'][1]
        n_Di = speciesStoi[singleSelection.value]['q_Di'][1]
        n_Do = speciesStoi[singleSelection.value]['q_Do'][1]
        solve = tuple()
        if dataset==False:
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ci$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Co$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Di$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Do$'+'  ($u$mol)'][0],)
        else:
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ci$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Co$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Di$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Do$'+'  ($u$mol)'][0],)
        
        bounds=[[],[]]
        for i in range(22):
            bounds[0].append(0)
            bounds[1].append(np.inf)

        def func(X, p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22):
            ai,ao,bi,bo,ci,co,di,do = X
            Ai,Ao,Bi,Bo,Ci,Co,Di,Do = ai**n_Ai, ao**n_Ao, bi**n_Bi, bo**n_Bo, ci**n_Ci, co**n_Co, di**n_Di, do**n_Do
            
            V_SS = Ai*Ao*Bi*Bo*(p1*Di*Co - p2*Do*Ci)/(p11*Ai*Ao*Bo*Di*Co+p12*Ai*Ao*Bo*Do*Ci+p13*Ai*Ao*Bo*Ci*Co+p14*Ai*Ao*Bi*Di*Co+p15*Ai*Ao*Bi*Do*Ci+p16*Ai*Ao*Bi*Ci*Co+p17*Ai*Ao*Bi*Bo*Do+p18*Ai*Ao*Bi*Bo*Co+p19*Ai*Ao*Bi*Bo*Di*Do+p20*Ai*Ao*Bi*Bo*Di*Co+p3*Ai*Ao*Bi*Bo*Di+p21*Ai*Ao*Bi*Bo*Do*Ci+p22*Ai*Ao*Bi*Bo*Ci*Co+p4*Ai*Ao*Bi*Bo*Ci+p5*Ao*Bi*Bo*Di*Co+p6*Ao*Bi*Bo*Do*Ci+p7*Ao*Bi*Bo*Ci*Co+p8*Ai*Bi*Bo*Di*Co*+p9*Ai*Bi*Bo*Do*Ci+p10*Ai*Bi*Bo*Ci*Co)

            return V_SS
        
        if dataset==False:
            popt, pcov = curve_fit(func,solve,[el.value for el in floatSSget if el.description=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)
        else:
            popt, pcov = curve_fit(func,solve,[floatSSget[el] for el in floatSSget.columns if el=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)

        pVals={}
        pVals[singleSelection.value]={}
        for i,el in enumerate(popt):
            pVals[singleSelection.value]['p'+str(i+1)]=el

        json_object = json.dumps(pVals, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
        with open("./temporary files/pVals.json", "w") as outfile:
            outfile.write(json_object)
            
            
            
            
            
    # This section works for 'Template 10'
    if singleSelection.value == 'Template 10':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        n_Ci = speciesStoi[singleSelection.value]['q_Ci'][1]
        n_Co = speciesStoi[singleSelection.value]['q_Co'][1]

        solve = tuple()
        if dataset==False:
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Ci$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='$q_Co$'+'  ($u$mol)'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='V_m'+'  (mV)'][0],)
        else:
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ai$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ao$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bi$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Bo$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Ci$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='$q_Co$'+'  ($u$mol)'][0],)
            solve = solve+([floatSSget[el] for el in floatSSget.columns if el=='V_m'+'  (mV)'][0],)

        bounds=[[],[]]
        for i in range(34):
            bounds[0].append(0)
            bounds[1].append(np.inf)

        def func(X, p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25,p26,p27,p28,p29,p30,p31,p32,p33,p34):
            ai,ao,bi,bo,ci,co,V_m = X
            Ai,Ao,Bi,Bo,Ci,Co = ai**n_Ai, ao**n_Ao, bi**n_Bi, bo**n_Bo, ci**n_Ci, co**n_Co
            F=96485
            R=8.31
            T=310
            if dataset==False:
                EXP = exp(F*V_m/(R*T))
            else:
                EXP = [exp(F*vm/(R*T)) for vm in V_m]
                
            V_SS = (p1*Ao*Bo*Co*EXP - p2*Ai*Bi*Ci)/(p11*Ao*Co *EXP + p12*Ai*Bi + p13*Ao*Bo+p14*Bi*Ci+p15*Ai*Bi*Ci+p16*Ai*Bo*Co*EXP+p17*Ao*Bo*Co*EXP+p18*Ao*Bo*Ci+p19*Ai*Bi*Co*EXP+p20*Ao*Bi*Ci+p3*Ci+p21*Ai*Bi*Bo*Co*EXP+p22*Ai*Bi*Bo*Ci+p23*Ao*Bo*Ci*Co*EXP+p24*Ai*Bi*Ci*Co*EXP+p25*Ai*Ao*Bi*Ci+p26*Ai*Ao*Bo*Co*EXP+p27*Ao*Bi*Bo*Co*EXP+p28*Ao*Bi*Bo*Ci+p29*Ai*Bi*Bo*Ci*Co*EXP+p30*Ai*Ao*Bi*Bo*Co*EXP+p4*Co*EXP+p31*Ai*Ao*Bi*Bo*Ci+p32*Ao*Bi*Bo*Ci*Co*EXP+p33*Ai*Ao*Bi*Bo*Ci*Co*EXP+p34+p5*Ai+p6*Ao+p7*Bo*Co*EXP+p8*Ai*Ci+p9*Ao*Ci+p10*Ai*Co*EXP)

            return V_SS

        if dataset==False:
            popt, pcov = curve_fit(func,solve,[el.value for el in floatSSget if el.description=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)
        else:
            popt, pcov = curve_fit(func,solve,[floatSSget[el] for el in floatSSget.columns if el=='V_SS'+'  ($u$mol/s)'][0], maxfev=3000000, bounds=bounds)


        pVals={}
        pVals[singleSelection.value]={}
        for i,el in enumerate(popt):
            pVals[singleSelection.value]['p'+str(i+1)]=el

        json_object = json.dumps(pVals, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
        with open("./temporary files/pVals.json", "w") as outfile:
            outfile.write(json_object)
                        
    return pVals


In [ ]:
# Define a function to convert a infix expression to a MathML string (temporary solution with limitations 1. no support for units 2. some MathML elements defined in CellML2.0 are not supported)
def eq_to_mathml(infix, Vss, ode=False):
    if ode == False:
        preforumla = '<apply> <eq/> <ci>'+ Vss + '</ci>'
    else:
        preforumla = '<apply> <eq/> <apply> <diff/> <bvar> <ci>'+ 't' + '</ci> </bvar> <ci>' + 'X' + '</ci> </apply> \n'
            
    postformula = '</apply>'
    
    p = libsbml.parseL3Formula (infix)
    mathstr = libsbml.writeMathMLToString (p)
    # remove the <math> tags in the mathML string
    mathstr = mathstr.replace ('<math xmlns="http://www.w3.org/1998/Math/MathML">', '')
    mathstr = mathstr.replace ('</math>', '')
    mathstr = mathstr.replace ('<?xml version="1.0" encoding="UTF-8"?>', '')
    # add left side of the equation
    mathstr1 = preforumla + mathstr + postformula
    mathstr2 = mathstr1.replace('\n','')
    mathstr3 = mathstr2.replace('type="integer"','cellml:units="dimensionless"')
    mathstr = mathstr3.replace('<cn>','<cn cellml:units="dimensionless">')
    
    return mathstr

In [ ]:
def singleModelCellmlGenerator(modelUnits,savedpVals,button_modelName):
    
    # Setting up the units

    Concentration = Units("concentration_unit")
    if modelUnits['Concentration'] ==1:
        Concentration.addUnit("mole")   # reference unit and built-in prefix
    else:
        Concentration.addUnit("mole", modelUnits['Concentration'])
        Concentration.addUnit('litre',modelUnits['Cell volume'],-1)

    flowRate = Units("flow_rate_unit")
    flowRate.addUnit("concentration_unit", 1)   # reference unit, prefix, exponent, multiplier
    if modelUnits['Flow rate'] ==1:
        flowRate.addUnit('second', 1, -1)
    else:
        flowRate.addUnit('second', modelUnits['Flow rate'], -1)
        
    membraneVoltageUnit = Units("mV")
    membraneVoltageUnit.addUnit('volt','milli')
    
    C_per_mol = Units("C_per_mol")
    C_per_mol.addUnit("coulomb", 1)   # reference unit, prefix, exponent, multiplier
    C_per_mol.addUnit('mole', 1, -1)
    
    J_per_K_per_mol = Units("J_per_K_per_mol")
    J_per_K_per_mol.addUnit("joule", 1)   # reference unit, prefix, exponent, multiplier
    J_per_K_per_mol.addUnit("kelvin", 1, -1)
    J_per_K_per_mol.addUnit("mole", 1, -1)
        
    time = Units('millisecond')
    time.addUnit('second', 'milli')
    

    
    # Setting up the maths
    file = open('./temporary files/speciesStoi.json')
    speciesStoi = json.load(file)

    if singleSelection.value == 'Template 3':
        equation = '(p1* q_Ao^n_Ao - p2* q_Ai^n_Ai)/ (p3 + p4*q_Ao^n_Ao + p5*q_Ai^n_Ai*q_Ao^n_Ao + p6*q_Ai^n_Ai)'
    if singleSelection.value == 'Template 4':
        equation = '(p1*q_Ao^n_Ao*q_Bi^n_Bi - p2*q_Ai^n_Ai*q_Bo^n_Bo)/ (p3*q_Ao^n_Ao + p4*q_Ai^n_Ai + p5*q_Bo^n_Bo + p6*q_Bi^n_Bi + p7*q_Ao^n_Ao*q_Bi^n_Bi + p8*q_Ai^n_Ai*q_Ao^n_Ao + p9*q_Ai^n_Ai*q_Bo^n_Bo + p10*q_Bo^n_Bo*q_Bi^n_Bi)'
    if singleSelection.value == 'Template 7':
        equation = '(p1*q_Ai^n_Ai*q_Bo^n_Bo*q_Co^n_Co - p2*q_Ao^n_Ao*q_Bi^n_Bi*q_Ci^n_Ci)/(p3*q_Ai^n_Ai + p4*q_Ao^n_Ao + p5*q_Bi^n_Bi + p6*q_Bo^n_Bo + p7*q_Ci^n_Ci + p8*q_Co^n_Co + p9*q_Ai^n_Ai*q_Ao^n_Ao + p10*q_Bi^n_Bi*q_Bo^n_Bo + p11*q_Ci^n_Ci*q_Co^n_Co + p12*q_Ai^n_Ai*q_Bo^n_Bo*q_Co^n_Co + p13*q_Bi^n_Bi*q_Ci^n_Ci*q_Ao^n_Ao)'
    if singleSelection.value == 'Template 11':
        equation = '(p1*q_Ai^n_Ai*q_Bo^n_Bo*q_Co^n_Co*q_Do^n_Do - p2*q_Ao^n_Ao*q_Bi^n_Bi*q_Ci^n_Ci*q_Di^n_Di)/(p3*q_Ai^n_Ai + p4*q_Ao^n_Ao + p5*q_Bi^n_Bi + p6*q_Bo^n_Bo + p7*q_Ci^n_Ci + p8*q_Co^n_Co + p9*q_Di^n_Di + p10*q_Do^n_Do + p11*q_Ai^n_Ai*q_Ao^n_Ao + p12*q_Bi^n_Bi*q_Bo^n_Bo + p13*q_Ci^n_Ci*q_Co^n_Co + p14*q_Di^n_Di*q_Do^n_Do + p15*q_Ai^n_Ai*q_Bo^n_Bo*q_Co^n_Co*q_Do^n_Do + p16*q_Bi^n_Bi*q_Ci^n_Ci*q_Ao^n_Ao*q_Di^n_Di)'    
    if singleSelection.value == 'Template 5':
        equation =  '(p1*q_Bo^n_Bo*q_Ao^n_Ao*exp(F*V_m/(R*T))-p2*q_Ai^n_Ai*q_Bi^n_Bi)/(p11*q_Bi^n_Bi*q_Ai^n_Ai*q_Ao^n_Ao*exp(F*V_m/(R*T))+p12*q_Bi^n_Bi*q_Bo^n_Bo*q_Ai^n_Ai*q_Ao^n_Ao*exp(F*V_m/(R*T))+p3*q_Bi^n_Bi*q_Bo^n_Bo+p4*q_Bo^n_Bo*q_Ai^n_Ai+p5*q_Bo^n_Bo*q_Ao^n_Ao*exp(F*V_m/(R*T))+p6*q_Bi^n_Bi*q_Ai^n_Ai+p7*q_Bi^n_Bi*q_Ao^n_Ao*exp(F*V_m/(R*T))+p8*q_Bi^n_Bi*q_Bo^n_Bo*q_Ai^n_Ai+p9*q_Bi^n_Bi*q_Bo^n_Bo*q_Ao^n_Ao*exp(F*V_m/(R*T))+p10*q_Bo^n_Bo*q_Ai^n_Ai*q_Ao^n_Ao*exp(F*V_m/(R*T)))'
    if singleSelection.value == 'Template 6':
        equation = '(q_Bo^n_Bo*q_Bi^n_Bi*(p1*q_Ao^n_Ao*exp(n_Bo*F*V_m/(R*T))-p2*q_Ai^n_Ai))/(p3*q_Bi^n_Bi*q_Bo^n_Bo + p4*q_Bi^n_Bi*q_Bo^n_Bo*exp(F*V_m/(R*T)) + p5*q_Bi^n_Bi*q_Bo^n_Bo*exp(n_Bo*F*V_m/(R*T))+ p6*q_Bo^n_Bo*q_Ai^n_Ai + p7*q_Bo^n_Bo*q_Ai^n_Ai*exp(F*V_m/(R*T)) + p8*q_Bo^n_Bo*q_Ai^n_Ai*exp(n_Bo*F*V_m/(R*T)) + p9*q_Bo^n_Bo*q_Ao^n_Ao*exp(n_Bo*F*V_m/(R*T))+ p10*q_Bi^n_Bi*q_Ai^n_Ai + p11*q_Bi^n_Bi*q_Ao^n_Ao+ p12*q_Bi^n_Bi*q_Ao^n_Ao*exp(F*V_m/(R*T)) + p13*q_Bi^n_Bi*q_Bo^n_Bo*q_Ai + p14*q_Bi^n_Bi*q_Bo^n_Bo*q_Ai^n_Ai*exp(F*V_m/(R*T))+ p15*q_Bi^n_Bi*q_Bo^n_Bo*q_Ai^n_Ai*exp(n_Bo*F*V_m/(R*T)) + p16*q_Bi^n_Bi*q_Bo^n_Bo*q_Ao^n_Ao*exp(n_Bo*F*V_m/(R*T)) + p17*q_Bi^n_Bi*q_Bo^n_Bo*q_Ao^n_Ao*exp(F*V_m/(R*T)) + p18*q_Bo^n_Bo*q_Ai^n_Ai*q_Ao^n_Ao*exp(n_Bo*F*V_m/(R*T)) + p19*q_Bi^n_Bi*q_Ai^n_Ai*q_Ao^n_Ao + p20*q_Bi^n_Bi*q_Bo^n_Bo*q_Ai^n_Ai*q_Ao^n_Ao*exp(n_Bo*F*V_m/(R*T)) + p21*q_Bi^n_Bi*q_Bo^n_Bo*q_Ai^n_Ai*q_Ao^n_Ao*exp(F*V_m/(R*T)))'
    if singleSelection.value == 'Template 8':
        equation = '(exp(n_Ao*F*V_m/(R*T))*(p1*q_Bo^n_Bo*q_Ao^n_Ao -p2*q_Bi^n_Bi*q_Ai^n_Ai))/(p11*q_Bi^n_Bi*q_Ai^n_Ai*q_Ao^n_Ao*exp(F*V_m/(R*T)) + p3*q_Bi^n_Bi*exp(F*V_m/(R*T)) + p4*q_Bi^n_Bi*exp(n_Ao*F*V_m/(R*T))+ p5*q_Bi^n_Bi*q_Ai^n_Ai*exp(n_Ao*F*V_m/(R*T))+ p6*q_Bo^n_Bo*q_Ai^n_Ai*exp(n_Ao*F*V_m/(R*T))+ p7*q_Bi^n_Bi*q_Ao^n_Ao*exp(F*V_m/(R*T))+ p8*q_Ai^n_Ai*exp(F*V_m/(R*T))+ p9*q_Ai^n_Ai*exp(n_Ao*F*V_m/(R*T))+ p10*q_Bi^n_Bi*q_Ao^n_Ao*exp(n_Ao*F*V_m/(R*T)))'
    if singleSelection.value == 'Template 1':
        equation = 'q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*(p1*q_Di^n_Di*q_Co^n_Co - p2*q_Do^n_Do*q_Ci^n_Ci)/(p11*q_Ai^n_Ai*q_Ao^n_Ao*q_Bo^n_Bo*q_Di^n_Di*q_Co^n_Co+p12*q_Ai^n_Ai*q_Ao^n_Ao*q_Bo^n_Bo*q_Do^n_Do*q_Ci^n_Ci+p13*q_Ai^n_Ai*q_Ao^n_Ao*q_Bo^n_Bo*q_Ci^n_Ci*q_Co^n_Co+p14*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Di^n_Di*q_Co^n_Co+p15*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Do^n_Do*q_Ci^n_Ci+p16*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Ci^n_Ci*q_Co^n_Co+p17*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Do^n_Do+p18*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Co^n_Co+p19*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Di^n_Di*q_Do^n_Do+p20*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Di^n_Di*q_Co^n_Co+p3*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Di^n_Di+p21*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Do^n_Do*q_Ci^n_Ci+p22*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Ci^n_Ci*q_Co^n_Co+p4*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Ci^n_Ci+p5*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Di^n_Di*q_Co^n_Co+p6*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Do^n_Do*q_Ci^n_Ci+p7*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Ci^n_Ci*q_Co^n_Co+p8*q_Ai^n_Ai*q_Bi^n_Bi*q_Bo^n_Bo*q_Di^n_Di*q_Co^n_Co+p9*q_Ai^n_Ai*q_Bi^n_Bi*q_Bo^n_Bo*q_Do^n_Do*q_Ci^n_Ci+p10*q_Ai^n_Ai*q_Bi^n_Bi*q_Bo^n_Bo*q_Ci^n_Ci*q_Co^n_Co)'
    if singleSelection.value == 'Template 10':
        equation = '(p1*q_Ao^n_Ao*q_Bo^n_Bo*q_Co^n_Co*exp(F*V_m/(R*T)) - p2*q_Ai^n_Ai*q_Bi^n_Bi*q_Ci^n_Ci)/(p11*q_Ao^n_Ao*q_Co^n_Co*exp(F*V_m/(R*T)) + p12*q_Ai^n_Ai*q_Bi^n_Bi + p13*q_Ao^n_Ao*q_Bo^n_Bo+p14*q_Bi^n_Bi*q_Ci^n_Ci+p15*q_Ai^n_Ai*q_Bi^n_Bi*q_Ci^n_Ci+p16*q_Ai^n_Ai*q_Bo^n_Bo*q_Co^n_Co*exp(F*V_m/(R*T))+p17*q_Ao^n_Ao*q_Bo^n_Bo*q_Co^n_Co*exp(F*V_m/(R*T))+p18*q_Ao^n_Ao*q_Bo^n_Bo*q_Ci^n_Ci+p19*q_Ai^n_Ai*q_Bi^n_Bi*q_Co^n_Co*exp(F*V_m/(R*T))+p20*q_Ao^n_Ao*q_Bi^n_Bi*q_Ci^n_Ci+p3*q_Ci^n_Ci+p21*q_Ai^n_Ai*q_Bi^n_Bi*q_Bo^n_Bo*q_Co^n_Co*exp(F*V_m/(R*T))+p22*q_Ai^n_Ai*q_Bi^n_Bi*q_Bo^n_Bo*q_Ci^n_Ci+p23*q_Ao^n_Ao*q_Bo^n_Bo*q_Ci^n_Ci*q_Co^n_Co*exp(F*V_m/(R*T))+p24*q_Ai^n_Ai*q_Bi^n_Bi*q_Ci^n_Ci*q_Co^n_Co*exp(F*V_m/(R*T))+p25*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Ci^n_Ci+p26*q_Ai^n_Ai*q_Ao^n_Ao*q_Bo^n_Bo*q_Co^n_Co*exp(F*V_m/(R*T))+p27*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Co^n_Co*exp(F*V_m/(R*T))+p28*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Ci^n_Ci+p29*q_Ai^n_Ai*q_Bi^n_Bi*q_Bo^n_Bo*q_Ci^n_Ci*q_Co^n_Co*exp(F*V_m/(R*T))+p30*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Co^n_Co*exp(F*V_m/(R*T))+p4*q_Co^n_Co*exp(F*V_m/(R*T))+p31*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Ci^n_Ci+p32*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Ci^n_Ci*q_Co^n_Co*exp(F*V_m/(R*T))+p33*q_Ai^n_Ai*q_Ao^n_Ao*q_Bi^n_Bi*q_Bo^n_Bo*q_Ci^n_Ci*q_Co^n_Co*exp(F*V_m/(R*T))+p34+p5*q_Ai^n_Ai+p6*q_Ao^n_Ao+p7*q_Bo^n_Bo*q_Co^n_Co*exp(F*V_m/(R*T))+p8*q_Ai^n_Ai*q_Ci^n_Ci+p9*q_Ao^n_Ao*q_Ci^n_Ci+p10*q_Ai^n_Ai*q_Co^n_Co*exp(F*V_m/(R*T)))'

          
    mathBody = eq_to_mathml(equation, 'V_SS', ode=False)
            
    fakeODE = eq_to_mathml('0', 'X', ode=True)    
    
    
    # Generate the CellML model, components, variables,...
    
    model = libcellml.Model()
    model.setName(button_modelName)
    
    model.addUnits(time)
    # model.addUnits(CellVolume)
    model.addUnits(Concentration)
    model.addUnits(flowRate)
    model.addUnits(membraneVoltageUnit)
    model.addUnits(C_per_mol)
    model.addUnits(J_per_K_per_mol)
    

    # Create a new component
    component = libcellml.Component()
    component.setName("main")

    # Add the component to the model
    model.addComponent(component)

    v = libcellml.Variable()
    v.setName('t')
    v.setUnits(time)
    v.setInterfaceType("public")
    component.addVariable(v)
    
    v = libcellml.Variable()
    v.setName('T')
    v.setUnits('kelvin')
    v.setInitialValue(310)
    v.setInterfaceType("public")
    component.addVariable(v)
    
    v = libcellml.Variable()
    v.setName('F')
    v.setUnits('C_per_mol')
    v.setInitialValue(96485)
    v.setInterfaceType("public")
    component.addVariable(v)
    
    v = libcellml.Variable()
    v.setName('R')
    v.setUnits('J_per_K_per_mol')
    v.setInitialValue(8.31)
    v.setInterfaceType("public")
    component.addVariable(v)

    # Create variables
    # Stoichiometries
    for q in speciesStoi[singleSelection.value]:
        v = Variable()
        v.setName(speciesStoi[singleSelection.value][q][0])
        v.setUnits('dimensionless')
        v.setInitialValue(speciesStoi[singleSelection.value][q][1])
        v.setInterfaceType("public")
        component.addVariable(v)
    
    # Concentration variables here,
    for variable in valueRequired[singleSelection.value]:
        if variable in [x for x in valueRequired1[singleSelection.value]]:
            v = Variable()
            v.setName('q_{}'.format(variable))
            v.setUnits(Concentration)
            v.setInitialValue([x.value for x in floatSSget if  x.description=='$q_{}$'.format(variable)+'  ($u$mol)'][0])
            v.setInterfaceType("public")
            component.addVariable(v)
        else:
            if variable=='V_SS':
                v = Variable()
                v.setName(variable)
                v.setUnits(flowRate)
                v.setInterfaceType("public")
                component.addVariable(v)
            if variable=='V_m':
                v = Variable()
                v.setName(variable)
                v.setUnits(membraneVoltageUnit)
                v.setInitialValue([x.value for x in floatSSget if  x.description==variable+'  (mV)'][0])
                v.setInterfaceType("public")
                component.addVariable(v)
        
        
    
    for variable in savedpVals[button_modelName]:
        v = Variable()
        v.setName(variable)
        v.setUnits('dimensionless')
        v.setInitialValue(savedpVals[button_modelName][variable])
        v.setInterfaceType("public")
        component.addVariable(v)
    
    # Setting up a fake ODE to run the code    
    v = Variable()
    v.setName('X')
    v.setUnits('dimensionless')
    v.setInitialValue('0')
    v.setInterfaceType("public")
    component.addVariable(v)
    
    # Assign IDs
    annotator = Annotator()
    annotator.setModel(model)

    annotator.clearAllIds()
    annotator.assignAllIds()

    # Create the equation
    math_header = '<math xmlns="http://www.w3.org/1998/Math/MathML" xmlns:cellml="http://www.cellml.org/cellml/2.0#">'
    math_footer = '</math>'

    component.setMath(math_header)
    component.appendMath(mathBody)
    component.appendMath(fakeODE)
    component.appendMath(math_footer)

    printer = libcellml.Printer()
    
    writeCellML(model,printer,button_modelName)
    
    return model
    
    


In [ ]:
# Write a model to cellml file, input: directory, model, output: cellml file
def writeCellML(model,printer,button_modelName):

    #     Write the serialised string to a file.
    write_file = open(button_modelName+".cellml", "w")
    write_file.write(printer.printModel(model))
    print("The model has been printed to: {}.cellml".format(model.name()))
    return 

In [ ]:
def xmlAnnot(model,savedAnnotations):
    # Define namespaces
    rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
    bqbiol = Namespace("http://biomodels.net/biology-qualifiers/")

    # Create RDF graph
    g = Graph()
    g.bind("rdf", rdf)
    g.bind("bqbiol", bqbiol)

    for compNum in range(model.componentCount()):
        for varNum in range(model.component(compNum).variableCount()):
            if model.component(compNum).variable(varNum).name() in ['q_Ao','q_Ai','q_Bo','q_Bi','q_Co','q_Ci','q_Do','q_Di']:
                to_be_annotated_var = model.component(compNum).variable(varNum).id()
                # Define RDF resources for the Concentrations
                Q1 = URIRef(button_modelName.value.strip('\n')+".cellml" + '#' + to_be_annotated_var)
                opb = URIRef("https://identifiers.org/opb/"+[x.replace('isVersionOf$','') for x in savedAnnotations[button_modelName.value.strip('\n')][model.component(compNum).variable(varNum).name()] if 'isVersionOf$' in x][0])
                for x in savedAnnotations[button_modelName.value.strip('\n')][model.component(compNum).variable(varNum).name()]:
                    if 'CHEBI' in x:
                        entity = URIRef("https://identifiers.org/chebi/"+[x.replace('entity$','') for x in savedAnnotations[button_modelName.value.strip('\n')][model.component(compNum).variable(varNum).name()] if 'entity$' in x][0])
                    if 'GO' in x:
                        entity = URIRef("http://purl.obolibrary.org/obo/"+[x.replace('entity$','') for x in savedAnnotations[button_modelName.value.strip('\n')][model.component(compNum).variable(varNum).name()] if 'entity$' in x][0])
                    if 'hasSourceParticipant$' in x:
                        sourceSink = URIRef("http://bime.uv.edu/semsim/"+[x.replace('hasSourceParticipant$','') for x in savedAnnotations[button_modelName.value.strip('\n')][model.component(compNum).variable(varNum).name()] if 'hasSourceParticipant$' in x][0])
                    if 'hasSinkParticipant$' in x:
                        sourceSink = URIRef("http://bime.uv.edu/semsim/"+[x.replace('hasSinkParticipant$','') for x in savedAnnotations[button_modelName.value.strip('\n')][model.component(compNum).variable(varNum).name()] if 'hasSinkParticipant$' in x][0])
                
                fma = URIRef("http://identifiers.org/fma/"+[x.replace('isPartOf$','') for x in savedAnnotations[button_modelName.value.strip('\n')][model.component(compNum).variable(varNum).name()] if 'isPartOf$' in x][0])
                
                # Add RDF triples to graph
                g.add((Q1, bqbiol.isVersionOf, opb))
                g.add((Q1, bqbiol.isVersionOf, entity))
                g.add((Q1, bqbiol.isPartOf, fma))
                g.add((Q1, bqbiol.sourceSink, sourceSink))

            if model.component(compNum).variable(varNum).name() == 'V_SS':
                to_be_annotated_var = model.component(compNum).variable(varNum).id()
                # Define RDF resources for the Concentrations
                Q1 = URIRef(button_modelName.value.strip('\n')+".cellml" + '#' + to_be_annotated_var)
                opb = URIRef("https://identifiers.org/opb/"+[x.replace('isVersionOf$','') for x in savedAnnotations[button_modelName.value.strip('\n')][model.component(compNum).variable(varNum).name()] if 'isVersionOf$' in x][0])

                g.add((Q1, bqbiol.isVersionOf, opb))

    # Serialize RDF graph as XML file
    xml_string = g.serialize(format='xml')

    # and save to a file
    with open('./{}.xml'.format(button_modelName.value.strip('\n')), 'w') as f:
        f.write(g.serialize(format='xml'))
        

In [2]:
def convertToCellml2(cellml_file,mainDir,newCellmlName):

    cellml_strict_mode = False
    if len(sys.argv) > 2:
        strict_mode = sys.argv[2]
        if strict_mode == 'false':
            cellml_strict_mode = False

    # if cellml_strict_mode:
    #     print('  Parsing files in STRICT mode (only CellML 2.0 models accepted)')
    # else:
    #     print('  Parsing files in NON-STRICT mode (any CellML models accepted)')

    model = parse_model(cellml_file, cellml_strict_mode)
    if validate_model(model) > 0:
        exit(-1)

    importer = resolve_imports(model, mainDir, cellml_strict_mode)
    if model.hasUnresolvedImports():
        print("unresolved imports?")
        exit(-2)

    if validate_model(model) > 0:
        print('Validation issues found')
        exit(-3)

    # print('Model was parsed, resolved, and validated without any issues.')

    # need a flattened model for analysing
    flat_model = flatten_model(model, importer)
    if validate_model(model) > 0:
        # print('Validation issues found in flattened model')
        exit(-4)

    # print('Model was flattened without any issues.')

    # this will report any issues that come up in analysing the model to prepare for code generation
    analysed_model = analyse_model(flat_model)

    annotator = Annotator()
    annotator.setModel(model)

    annotator.clearAllIds()
    annotator.assignAllIds()
    model_string = print_model(model)

    # and save the updated model to a new file - note, we need the model filename for making our annotations later
    model_file = './savedModels/PMR/{}.cellml'.format(newCellmlName)
    with open(model_file, 'w') as f:
        f.write(model_string)
    return model

In [ ]:
def pmrSearching(name):
    sparqlendpoint = 'https://models.physiomeproject.org/pmr2_virtuoso_search'
    sparql = SPARQLWrapper(sparqlendpoint)

    def search_entity(terms):
        query = """SELECT ?graph ?Model_entity WHERE {{ GRAPH ?graph {{ ?Model_entity ?p ?o FILTER REGEX(LCASE(STR(?Model_entity)), '{terms}')}}}}""".format(terms=terms)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        graphs = sparql.query().convert()
        return graphs

    def search_model(terms):
        terms = terms.lower()
        entities = search_entity(terms)
        model = set()
        for entity in entities['results']['bindings']:
            workspace = entity['graph']['value']
            cellml = entity['Model_entity']['value'].split('#')[0]
            if not cellml.startswith('http') and terms in cellml.lower():
                model.update([workspace+'/rawfile/HEAD/'+cellml])
        return list(model)

    pmrModel = search_model(name)

    return pmrModel

In [ ]:
def RDFpmrSearching(name):
    sparqlendpoint = 'https://models.physiomeproject.org/pmr2_virtuoso_search'
    sparql = SPARQLWrapper(sparqlendpoint)
    
    def search_entity(terms):
        query = """SELECT ?graph ?s ?p ?o WHERE {{ GRAPH ?graph {{ ?s ?p ?o FILTER REGEX(LCASE(STR(?s)), '{terms}')}}}}""".format(terms=terms)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        graphs = sparql.query().convert()
        return graphs
    
    def search_model(terms):
        terms = terms.lower()
        entities = search_entity(terms)
        results = {}
        for entity in entities['results']['bindings']:
            workspace = entity['graph']['value']
            if workspace not in results:
                results[workspace] = []
            results[workspace] += [(entity['s']['value'], entity['p']['value'], entity['o']['value'])]
        return results
    
    return search_model(name)

In [ ]:
def q_to_mathml(infix, speciesNum, ode=False):
    if ode == False:
        preforumla = '<apply> <eq/> <ci>'+ 'q'+str(speciesNum)+ '</ci>'
    else:
        preforumla = '<apply> <eq/> <apply> <diff/> <bvar> <ci>'+ 't' + '</ci> </bvar> <ci>' + 'q'+str(speciesNum) + '</ci> </apply> \n'
        
    postformula = '</apply>'
    p = libsbml.parseL3Formula (infix)
    mathstr = libsbml.writeMathMLToString (p)
    # remove the <math> tags in the mathML string
    mathstr = mathstr.replace ('<math xmlns="http://www.w3.org/1998/Math/MathML">', '')
    mathstr = mathstr.replace ('</math>', '')
    mathstr = mathstr.replace ('<?xml version="1.0" encoding="UTF-8"?>', '')
    # add left side of the equation
    mathstr1 = preforumla + mathstr + postformula
    mathstr2 = mathstr1.replace('\n','')
    mathstr3 = mathstr2.replace('type="integer"','cellml:units="dimensionless"')
    mathstr = mathstr3.replace('<cn>','<cn cellml:units="dimensionless">')
    
    
    return mathstr

In [ ]:
def pythonCreator(mainDir,new_cellml_file):
    # if __name__ == '__main__':

    # STEP 1
    # Parse the model from a CellML file.

    # Create a libCellML Parser, and use it to parse the fileContents
    # string and convert it into a CellML Model structure.
    read_file = open('{}{}.cellml'.format(mainDir,new_cellml_file), "r")
    parser = Parser()
    model = parser.parseModel(read_file.read())

    # STEP 2
    # Resolve any import dependencies (if present) in the model.

    if(model.hasUnresolvedImports()) :

        # Create an Importer instance.
        importer = Importer()

        # Submit the model to the importer and the absolute location 
        # against which the import reference paths will be resolved.
        importer.resolveModelImports(model, "resources/")
        print_issues_to_terminal(importer)

        # Print a list of sources that this model requires. This list will 
        # be empty after the model has been flattened.
        print_import_dependencies(model)

        # Retrieve a "flattened" (ie: import-free) model from the importer,
        # and use it to over-write the current model.
        model = importer.flattenModel(model)


    # STEP 3
    # Validate the model: check for syntactic and semantic errors.

    # Create a Validator instance and pass the model for checking.
    validator = Validator()
    validator.validateModel(model)

    # STEP 4
    # Analyse a model: check for mathematical and modelling errors.
    analyser = Analyser()
    analyser.analyseModel(model)
    
    
    if validate_model(model) > 0:
        print('Validation issues found')
        exit(-1)

    if model.hasUnresolvedImports():
        print("unresolved imports?")
        exit(-2)


    print('Model was parsed, resolved, and validated without any issues.')

    # STEP 5
    # Generate runnable code in other language formats for this model.

    # Create a Generator instance.  Note that by default this is the C language.
    generator = Generator()
    generator.setModel(analyser.model())
    generator.interfaceCode()
    generator.implementationCode()

    # Pass the generator the analysed model for processing.
    # generator.processModel(analyser.model())


    # STEP 6

    # If required, change the generator profile to Python and reprocess the model.
    profile = GeneratorProfile(GeneratorProfile.Profile.PYTHON)
    generator.setProfile(profile)
    # generator.processModel(model)

    # Retrieve and write the implementation code (*.py) to a file.
    write_file = open('{}.py'.format(new_cellml_file), "w")
    write_file.write(generator.implementationCode())
    write_file.close()

    # END


In [ ]:
def PMRmodelComposition():
    mainDir = os.path.dirname("./savedModels/PMR/")
    folder_path = "./savedModels/PMR"
    extensions = (".cellml")
    
    global newCellmlNames
    cellml_files = []
    newCellmlNames = []

    # Loop through files in folder
    for file in os.listdir(folder_path):
        # Check if the file has a specific extension
        if file.endswith(extensions):
            # Add the file to the list
            cellml_files.append(os.path.join(folder_path, file))
    
    for file in cellml_files:
        filename = os.path.basename(file).split('.')[0]
        newCellmlNames.append(filename)

    newModels = {}
    for cellml_file,newCellmlName in zip(cellml_files,newCellmlNames):
        newModels[newCellmlName]=convertToCellml2(cellml_file,mainDir,newCellmlName)


    # global savedAnnotations
    savedAnnotations={}

    for key in xmlGrouped:
        newKey = key.replace('.cellml','')
        savedAnnotations[newKey] = xmlGrouped[key]
        
        
    global Species
    Species = []
    for transporterName in savedAnnotations:
        for entity in savedAnnotations[transporterName]:
            for predicate,obj in savedAnnotations[transporterName][entity]:
                if 'https://identifiers.org/opb/OPB_00340' in obj: # identifying the concentration of chemicals
                    Species.append([transporterName,savedAnnotations[transporterName][entity]])
    
    global speciesNoDuplicate
    speciesNoDuplicate = []

    for i, element in enumerate(Species):
        # remove 'sink' and 'source' strings from the element and sort the list
        clean_element = sorted([x for x in element[1] if x[1] not in ["http://bime.uv.edu/semsim/Source", "http://bime.uv.edu/semsim/Sink"]])
        # check if the element is unique
        is_unique = True
        for j in range(i+1, len(Species)):
            if sorted([x for x in Species[j][1] if x[1] not in ["http://bime.uv.edu/semsim/Source", "http://bime.uv.edu/semsim/Sink"]]) == clean_element:
                is_unique = False
                break
        if is_unique:
            speciesNoDuplicate.append(clean_element)


    for i,species in enumerate(speciesNoDuplicate):
        speciesNoDuplicate[i] = speciesNoDuplicate[i]+['q'+str(i)]


    V = []
    for transporterName in savedAnnotations:
        for entity in savedAnnotations[transporterName]:
            for predicate,obj in savedAnnotations[transporterName][entity]:
                if 'https://identifiers.org/opb/OPB_00592' in obj: # identifying the SS fluxes
                    V.append([transporterName,savedAnnotations[transporterName][entity],'V_SS'])


    M=np.zeros((len(speciesNoDuplicate),len(V)))
    coefficient=0
    for i,species in enumerate(speciesNoDuplicate):
        for element in Species:
            if all(el in element[1] for el in species[:-1]):
                for y in element[1]:
                    if 'http://bime.uv.edu/semsim/Sink' in y[1]:
                        coefficient = +1
                    if 'http://bime.uv.edu/semsim/Source' in y[1]:
                        coefficient = -1
                    for j,fluxinfo in enumerate(V):
                        if element[0]==fluxinfo[0]:
                            M[i][j] = coefficient


    odeRightSide={}
    for i in range(M.shape[0]):
        odeRightSide[i] = ''
        for j in range(len(V)):
            if M[i][j] > 0: 
                odeRightSide[i] += '+'+str(M[i][j]) +'*'+V[j][0].replace('.cellml','')+'_'+V[j][2]
            if M[i][j] < 0: 
                odeRightSide[i] += str(M[i][j]) +'*'+V[j][0].replace('.cellml','')+'_'+V[j][2]

    cellMLRef = {}
    for modelName in newModels:
        for compNum in range(newModels[modelName].componentCount()):
            for varNum in range(newModels[modelName].component(compNum).variableCount()):
                for flux in V:
                    if flux[0] ==  modelName:
                        for ID,value in savedAnnotations[modelName].items():
                            if all(el in flux[1][0] for el in value[0]) and ID==newModels[modelName].component(compNum).variable(varNum).id(): 
                                cellMLRef[modelName+'_'+newModels[modelName].component(compNum).variable(varNum).name()]=[]
                                cellMLRef[modelName+'_'+newModels[modelName].component(compNum).variable(varNum).name()].append(newModels[modelName].component(compNum).variable(varNum).units())
                                cellMLRef[modelName+'_'+newModels[modelName].component(compNum).variable(varNum).name()].append(newModels[modelName].component(compNum).variable(varNum).initialValue())
                                cellMLRef[modelName+'_'+newModels[modelName].component(compNum).variable(varNum).name()].append(newModels[modelName].component(compNum).variable(varNum).id())

                for speciesNum,species in enumerate(speciesNoDuplicate):
                    for key,value in savedAnnotations[modelName].items():
                        if all(el in value[:-1] for el in species[:-1]) and key==newModels[modelName].component(compNum).variable(varNum).id():
                            if 'q'+str(speciesNum) not in cellMLRef:
                                cellMLRef['q'+str(speciesNum)]=[]
                                
                                cellMLRef['q'+str(speciesNum)].append(newModels[modelName].component(compNum).variable(varNum).units())
                                cellMLRef['q'+str(speciesNum)].append(newModels[modelName].component(compNum).variable(varNum).initialValue())
                                cellMLRef['q'+str(speciesNum)].append(newModels[modelName].component(compNum).variable(varNum).id())



                if (modelName+'_'+newModels[modelName].component(compNum).variable(varNum).name() not in cellMLRef):
                    cellMLRef[modelName+'_'+newModels[modelName].component(compNum).variable(varNum).name()]=[]
                    cellMLRef[modelName+'_'+newModels[modelName].component(compNum).variable(varNum).name()].append(newModels[modelName].component(compNum).variable(varNum).units())
                    cellMLRef[modelName+'_'+newModels[modelName].component(compNum).variable(varNum).name()].append(newModels[modelName].component(compNum).variable(varNum).initialValue())
                    cellMLRef[modelName+'_'+newModels[modelName].component(compNum).variable(varNum).name()].append(newModels[modelName].component(compNum).variable(varNum).id())

    
    unitNames = []
    for variable in cellMLRef:
        if list(cellMLRef[variable])[0].name() not in unitNames: 
            unitNames.append(list(cellMLRef[variable])[0].name())
    common=[]
    modelUnits= []
    for unit in unitNames:
        for variable in cellMLRef:
            if list(cellMLRef[variable])[0].name() == unit and list(cellMLRef[variable])[0].name() not in common: 
                modelUnits.append(list(cellMLRef[variable])[0])
                common.append(list(cellMLRef[variable])[0].name())
                
                
    var_q=[]
    for modelName in newModels:
        for x in [x for x in cellMLRef.keys() if modelName+'_' in x ]:
            if modelName+'_' in x and cellMLRef[x][-1] in savedAnnotations[modelName].keys() and cellMLRef[x][0].name()=='concentration_unit':
                # print(modelName,x,cellMLRef[x][-1])
                for species in speciesNoDuplicate:
                    if all(el in savedAnnotations[modelName][cellMLRef[x][-1]] for el in species[:-1]):
                        var_q.append((x,species[-1]))
                        
    mathmlEquations={}
    for modelName,model in newModels.items():
        for flux in V:
            if flux[0] == modelName and flux[1][0][1] == 'https://identifiers.org/opb/OPB_00592': # is flux
                fluxID = flux[2]
        component = model.component(0)
        X1 = component.math().replace('<math xmlns="http://www.w3.org/1998/Math/MathML" xmlns:cellml="http://www.cellml.org/cellml/2.0#">','')
        X2 = X1.replace('<math xmlns="http://www.w3.org/1998/Math/MathML">','') 
        X3 = X2.replace('</math>','')
        X4 = X3.replace('\n','')
        X5 = X4.replace(' ','')
        X6 = X5.replace('<apply><eq/><ci>'+fluxID+'</ci>','<apply><eq/><ci>'+modelName+'_'+fluxID+'</ci>')  # FIND IT BY CHECKING THE ANNOTATIONS
        X = X6.replace('cncellml','cn cellml')

        for compNum in range(newModels[modelName].componentCount()):
            for varNum in range(newModels[modelName].component(compNum).variableCount()):
                X = X.replace('>'+newModels[modelName].component(compNum).variable(varNum).name()+'<','>'+modelName+'_'+newModels[modelName].component(compNum).variable(varNum).name()+'<')


        mathmlEquations[modelName] = X



    preMath=''
    newMath =''
    for file_name in newModels:
        preMath += mathmlEquations[file_name]

    for speciesNum in odeRightSide: # equivalent to speciesNoDuplicate   
        newMath += q_to_mathml(odeRightSide[speciesNum], speciesNum, ode=True)
    mathBody = preMath + newMath
    
    
    model = libcellml.Model()
    model.setName("CompositeModel")
    


    for unit in modelUnits:
        model.addUnits(unit)


    # Create a new component
    component = libcellml.Component()
    component.setName("MyComponent")

    # Add the component to the model
    model.addComponent(component)
    
    
    # Add time variable
    time = Units('millisecond')
    time.addUnit('second', 'milli')
    
    v = libcellml.Variable()
    v.setName('t')
    v.setUnits(time)
    v.setInterfaceType("public")
    component.addVariable(v)

    # Create variables
    used=[]
    for variable in cellMLRef:
        if variable not in [modelName+'_t' for modelName in newModels.keys()] and variable not in [modelName+'_X' for modelName in newModels.keys()]:
            if variable not in [x[0] for x in var_q] and variable not in [x[1] for x in var_q]:
                v = libcellml.Variable()
                v.setName(variable)
                v.setUnits(list(cellMLRef[variable])[0])
                v.setInitialValue(list(cellMLRef[variable])[1])
                v.setInterfaceType("public")
                component.addVariable(v)
            else:
                for var,q in var_q:
                    if var == variable and q not in used:
                        v = libcellml.Variable()
                        v.setName(q)
                        v.setUnits(list(cellMLRef[q])[0])
                        v.setInitialValue(list(cellMLRef[variable])[1])
                        v.setInterfaceType("public")
                        component.addVariable(v)
                        used.append(q)


    # Create the equation
    math_header = '<math xmlns="http://www.w3.org/1998/Math/MathML" xmlns:cellml="http://www.cellml.org/cellml/2.0#">'
    math_footer = '</math>'



    for modelName,modelinfo in newModels.items(): 
        mathBody = mathBody.replace('<ci>'+modelName+'_'+'t'+'</ci>','<ci>'+'t'+'</ci>')
        mathBody = mathBody.replace('<apply><eq/><apply><diff/><bvar><ci>t</ci></bvar><ci>'+modelName+'_'+'X</ci></apply><cn cellml:units="dimensionless">0</cn></apply>','')

        
    for var,q in var_q:
        mathBody = mathBody.replace('<ci>'+var+'</ci>','<ci>'+q+'</ci>')


    component.setMath(math_header)
    component.appendMath(mathBody)
    component.appendMath(math_footer)

    annotator = Annotator()
    annotator.setModel(model)

    annotator.clearAllIds()
    annotator.assignAllIds()

    # Print the model
    printer = libcellml.Printer()
    # print(printer.printModel(model))

    writeCellML(model,printer,'CompositeModel')
    
    return

In [55]:
def graphPlot(newCellmlNames,Species,speciesNoDuplicate,figSize):        
    entities = [('H+','CHEBI:15378'),('HCO3-','CHEBI:17544'), ('K+','CHEBI:29103'), ('Na+','CHEBI:29101'), ('Mg2+','CHEBI:18420'), ('Cl-','CHEBI:17996'), ('Ca2+','CHEBI:29108'), ('Fe2+','CHEBI:29033'), ('P','CHEBI:30207')]
    locations = [('Extracellular environment','fma70022'), ('Cytosol of stem cell','fma260697'), ('Cytosol of neuron','fma226054'), ('Cytosol of connective tissue cell','fma260699'),('Cytosol of skeletal muscle fiber','fma226052'),('Cytosol of cardiac myocyte','fma226050'),('Cytosol of striated muscle cell','fma226048'),('Cytosol of smooth muscle cell','fma226046'),('Cytosol of muscle cell','fma226044'), ('Cytosol of hemal cell','fma260695'), ('Cytosol of epithelial cell','fma260691')]

    connections=[]
    SourcesTop=[]
    SinksBelow=[]
    for modelName in newCellmlNames:
        for contentSpecies in Species:   
            if contentSpecies[0]==modelName:
                for x in contentSpecies[1]:
                    if 'isPartOf' in x[0]:
                        if 'fma70022' in x[1]:
                            SourcesTop.append([el[0] for el in entities if el[1] in [y[1] for y in contentSpecies[1] if y[0]=='http://biomodels.net/biology-qualifiers/isVersionOf' and  ('https://identifiers.org/chebi' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0]+' in '+[el[0] for el in locations if el[1] in [y[1] for y in contentSpecies[1] if y[0]=='http://biomodels.net/biology-qualifiers/isPartOf' and  ( 'http://identifiers.org/fma' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0])
                        else:
                            SinksBelow.append([el[0] for el in entities if el[1] in [y[1] for y in contentSpecies[1] if y[0]=='http://biomodels.net/biology-qualifiers/isVersionOf' and  ('https://identifiers.org/chebi' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0]+' in '+[el[0] for el in locations if el[1] in [y[1] for y in contentSpecies[1] if y[0]=='http://biomodels.net/biology-qualifiers/isPartOf' and  ( 'http://identifiers.org/fma' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0])

                    if x[1] == 'http://bime.uv.edu/semsim/Source':
                        A_disp = [el[0] for el in entities if el[1] in [y[1] for y in contentSpecies[1] if y[0]=='http://biomodels.net/biology-qualifiers/isVersionOf' and  ('https://identifiers.org/chebi' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0]+' in '+[el[0] for el in locations if el[1] in [y[1] for y in contentSpecies[1] if y[0]=='http://biomodels.net/biology-qualifiers/isPartOf' and  ( 'http://identifiers.org/fma' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0]
                        connections.append((A_disp,modelName))

                    if x[1] == 'http://bime.uv.edu/semsim/Sink':
                        B_disp = [el[0] for el in entities if el[1] in [y[1] for y in contentSpecies[1] if y[0]=='http://biomodels.net/biology-qualifiers/isVersionOf' and  ('https://identifiers.org/chebi' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0]+' in '+[el[0] for el in locations if el[1] in [y[1] for y in contentSpecies[1] if y[0]=='http://biomodels.net/biology-qualifiers/isPartOf' and  ( 'http://identifiers.org/fma' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0]
                        connections.append((modelName,B_disp))

    speciesNoDuplicateText=[]                
    for species in speciesNoDuplicate:
        speciesNoDuplicateText.append([el[0] for el in entities if el[1] in [y[1] for y in species if y[0]=='http://biomodels.net/biology-qualifiers/isVersionOf' and  ('https://identifiers.org/chebi' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0]+' in '+[el[0] for el in locations if el[1] in [y[1] for y in species if y[0]=='http://biomodels.net/biology-qualifiers/isPartOf' and  ('http://identifiers.org/fma' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0])

    # Create an empty graph
    G = nx.DiGraph()

    # Draw the graph

    plt.figure(figsize=(figSize, figSize)) 

    fixed_positions  = {key: ((figSize/len(newCellmlNames))+i*(figSize/len(newCellmlNames)),0) for key, i in zip(newCellmlNames, range(len(newCellmlNames)))}
    source_positions = {key: ((figSize/len(SourcesTop))+i*(figSize/len(SourcesTop)),3) for key, i in zip(SourcesTop, range(len(SourcesTop)))}
    sink_positions = {key: ((figSize/len(SinksBelow))+i*(figSize/len(SinksBelow)),-3) for key, i in zip(SinksBelow, range(len(SinksBelow)))}


    # Combine the positions
    positions = {**fixed_positions, **source_positions, **sink_positions}
        
    compartments={}
    for species in speciesNoDuplicate:
        compartment = [el[0] for el in locations if el[1] in [y[1] for y in species if y[0]=='http://biomodels.net/biology-qualifiers/isPartOf' and  ('http://identifiers.org/fma' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0]
        if compartment not in compartments.keys():
            compartments[compartment]=[]
            compartments[compartment].append([el[0] for el in entities if el[1] in [y[1] for y in species if y[0]=='http://biomodels.net/biology-qualifiers/isVersionOf' and  ('https://identifiers.org/chebi' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0]+' in '+[el[0] for el in locations if el[1] in [y[1] for y in species if y[0]=='http://biomodels.net/biology-qualifiers/isPartOf' and  ( 'http://identifiers.org/fma' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0])
        else:
            compartments[compartment].append([el[0] for el in entities if el[1] in [y[1] for y in species if y[0]=='http://biomodels.net/biology-qualifiers/isVersionOf' and  ('https://identifiers.org/chebi' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0]+' in '+[el[0] for el in locations if el[1] in [y[1] for y in species if y[0]=='http://biomodels.net/biology-qualifiers/isPartOf' and  ( 'http://identifiers.org/fma' in y[1] or  'https://identifiers.org/GO' in y[1])][0]][0])
    
    # Create a color map with a unique color for each compartment
    compartment_colors = {compartment: color for compartment, color in zip(compartments.keys(), cm.tab20(range(len(compartments.keys()))))}

    
    # Add nodes
    for compartment, nodes in compartments.items():
        G.add_nodes_from(nodes, compartment=compartment)
    G.add_nodes_from(newCellmlNames, compartment=compartment)
        
 
    # Draw the graph with different colors for compartments
    # for compartment,nodes in compartments.items():
    #     nx.draw_networkx_nodes(G, positions, nodelist=nodes, node_shape='o', node_color=compartment_colors[compartment], node_size=3000)
    for node, node_attrs in G.nodes(data=True):
        compartment = node_attrs['compartment']
        nx.draw_networkx_nodes(G, positions, nodelist=[node],
                               node_shape='o', node_color=compartment_colors[compartment], node_size=3000)


    
    # Draw transporters (model names)
    nx.draw_networkx_nodes(G, positions, nodelist=newCellmlNames, node_shape='H', node_color='lightgrey', node_size=5000)    
        
        
    # Add edges
    G.add_edges_from(connections)
    
    edge_arrows = nx.draw_networkx_edges(G, positions, edge_color='grey',  arrowsize=30, arrowstyle="-|>", width=0.5, arrows=True)#,  connectionstyle='angle,angleA=0,angleB=90')

    for arrow in edge_arrows:
        arrow.set_zorder(2)


    labels = nx.draw_networkx_labels(G, positions, font_color='black', font_size=10)#, font_weight='bold')

    # Adjust the positions of labels to avoid overlap
    adjust_text(texts=list(labels.values()), autoalign='xy', va='center')


    # Show the graph
    plt.show()

In [7]:
def templateGraphPlot(Type,figSize):    
    types = ['Template 3', 'Template 4', 'Template 7', 'Template 11', 'Template 5', 'Template 6', 'Template 8', 'Template 1', 'Template 10']

    elements = dict((el,[]) for el in types)
    elements['Template 3']=['$q_{A_o}$','$q_{A_i}$']
    elements['Template 4']=['$q_{A_o}$','$q_{A_i}$','$q_{B_o}$','$q_{B_i}$']
    elements['Template 7']=['$q_{A_o}$','$q_{A_i}$','$q_{B_o}$','$q_{B_i}$','$q_{C_o}$','$q_{C_i}$']
    elements['Template 11']=['$q_{A_o}$','$q_{A_i}$','$q_{B_o}$','$q_{B_i}$','$q_{C_o}$','$q_{C_i}$', '$q_{D_o}$','$q_{D_i}$']
    elements['Template 5']=['$q_{A_o}$','$q_{A_i}$','$q_{B_o}$','$q_{B_i}$','$V_m$']
    elements['Template 6']=['$q_{A_o}$','$q_{A_i}$','$q_{B_o}$','$q_{B_i}$','$V_m$']
    elements['Template 8']=['$q_{A_o}$','$q_{A_i}$','$q_{B_o}$','$q_{B_i}$','$V_m$']
    elements['Template 1']=['$q_{A_o}$','$q_{A_i}$','$q_{B_o}$','$q_{B_i}$','$q_{C_o}$','$q_{C_i}$', '$q_{D_o}$','$q_{D_i}$']
    elements['Template 10']=['$q_{A_o}$','$q_{A_i}$','$q_{B_o}$','$q_{B_i}$','$q_{C_o}$','$q_{C_i}$','$V_m$']
    
    
    connections= dict((el,[]) for el in types)
    connections['Template 3']=[('$q_{A_o}$','Template 3'),('Template 3','$q_{A_i}$')]
    connections['Template 4']=[('$q_{A_o}$','Template 4'),('Template 4','$q_{A_i}$'),('$q_{B_i}$','Template 4'),('Template 4','$q_{B_o}$')]
    connections['Template 7']=[('$q_{A_i}$','Template 7'),('Template 7','$q_{A_o}$'),('$q_{B_o}$','Template 7'),('Template 7','$q_{B_i}$'),('$q_{C_o}$','Template 7'),('Template 7','$q_{C_i}$')]
    connections['Template 11']=[('$q_{A_i}$','Template 11'),('Template 11','$q_{A_o}$'),('$q_{B_o}$','Template 11'),('Template 11','$q_{B_i}$'),('$q_{C_o}$','Template 11'),('Template 11','$q_{C_i}$'),('$q_{D_o}$','Template 11'),('Template 11','$q_{D_i}$')]
    connections['Template 5']=[('$q_{A_o}$','Template 5'),('Template 5','$q_{A_i}$'),('$q_{B_o}$','Template 5'),('Template 5','$q_{B_i}$'),('$V_m$','$q_{A_o}$')]
    connections['Template 6']=[('$q_{A_o}$','Template 6'),('Template 6','$q_{A_i}$'),('$q_{B_o}$','Template 6'),('Template 6','$q_{B_i}$'),('$V_m$','$q_{B_o}$')]
    connections['Template 8']=[('$q_{A_o}$','Template 8'),('Template 8','$q_{A_i}$'),('$q_{B_o}$','Template 8'),('Template 8','$q_{B_i}$'),('$V_m$','$q_{A_o}$')]
    connections['Template 1']=[('$q_{A_o}$','Template 1'),('Template 1','$q_{A_i}$'),('$q_{B_o}$','Template 1'),('Template 1','$q_{B_i}$'),('$q_{C_o}$','Template 1'),('Template 1','$q_{C_i}$'),('$q_{D_i}$','Template 1'),('Template 1','$q_{D_o}$')]
    connections['Template 10']=[('$q_{A_o}$','Template 10'),('Template 10','$q_{A_i}$'),('$q_{B_o}$','Template 10'),('Template 10','$q_{B_i}$'),('$q_{C_o}$','Template 10'),('Template 10','$q_{C_i}$'),('$V_m$','$q_{C_o}$')]
        
    
    SourcesTop=[]
    SinksBelow=[]
    membrane=[]
    
    for x in elements[Type]:   
        if x in ['$q_{A_i}$','$q_{B_i}$','$q_{C_i}$','$q_{D_i}$']:
            SinksBelow.append(x)
            # connections.append((Type,x))    
        elif x == '$V_m$':
            membrane.append(x)
            # connections.append((x,Type))
        else:
            SourcesTop.append(x)
            # connections.append((x,Type))


    # Create an empty graph
    G = nx.DiGraph()

    # Add nodes

    G.add_nodes_from(elements[Type])
    G.add_nodes_from([Type])
    try:
        if membrane:
            G.add_nodes_from(membrane)
    except:
        pass


    # Add edges
    G.add_edges_from(connections[Type])


    # Draw the graph

    plt.figure(figsize=(figSize, figSize)) 

    fixed_positions  = {key: ((figSize/len([Type]))+i*(figSize/len([Type])),0) for key, i in zip([Type], range(len([Type])))}
    source_positions = {key: ((figSize/len(SourcesTop+membrane))+i*(figSize/len(SourcesTop+membrane)),3) for key, i in zip(SourcesTop+membrane, range(len(SourcesTop+membrane)))}
    sink_positions = {key: ((figSize/len(SinksBelow))+i*(figSize/len(SinksBelow)),-3) for key, i in zip(SinksBelow, range(len(SinksBelow)))}


    # Combine the positions
    positions = {**fixed_positions, **source_positions, **sink_positions}


    # Draw nodes with custom shapes
    for node in [Type]:
        nx.draw_networkx_nodes(G, positions, nodelist=[node], node_shape='H', node_color='orange', node_size=5000)
    for node in SourcesTop+SinksBelow:
        nx.draw_networkx_nodes(G, positions, nodelist=[node], node_shape='o', node_color='lightblue',   node_size=3000)
    for node in membrane:
        nx.draw_networkx_nodes(G, positions, nodelist=[node], node_shape='v', node_color='yellowgreen',   node_size=3000)


    edge_arrows = nx.draw_networkx_edges(G, positions, edge_color='gray',  arrowsize=30, arrowstyle="-|>", width=0.5, arrows=True)

    for arrow in edge_arrows:
        arrow.set_zorder(2)


    labels = nx.draw_networkx_labels(G, positions, font_color='black', font_size=10)

    # Adjust the positions of labels to avoid overlap
    adjust_text(texts=list(labels.values()), autoalign='y', va='center')


    # Show the graph
    plt.show()
    


In [ ]:
def annotate(key,element,selectedAnnotations):

    # Create a dropdown widget for selecting the search column
    column_names_left = [('H+','entity$CHEBI:15378'),('HCO3-','entity$CHEBI:17544'), ('K+','entity$CHEBI:29103'), ('Na+','entity$CHEBI:29101'), ('Mg2+','entity$CHEBI:18420'), ('Cl-','entity$CHEBI:17996'), ('Ca2+','entity$CHEBI:29108'), ('Fe2+','entity$CHEBI:29033'), ('P','entity$CHEBI:30207')]
    column_names_widget_left = widgets.Dropdown(
        options = column_names_left,
        style={'description_width': 'initial'},
        disabled = False,
        layout = Layout(width='400px', height='60px')
    )

    column_names_middle = [('Extracellular environment','isPartOf$fma70022'), ('Cytosol of stem cell','isPartOf$fma260697'), ('Cytosol of neuron','isPartOf$fma226054'), ('Cytosol of connective tissue cell','isPartOf$fma260699'),('Cytosol of skeletal muscle fiber','isPartOf$fma226052'),('Cytosol of cardiac myocyte','isPartOf$fma226050'),('Cytosol of striated muscle cell','isPartOf$fma226048'),('Cytosol of smooth muscle cell','isPartOf$fma226046'),('Cytosol of muscle cell','isPartOf$fma226044'), ('Cytosol of hemal cell','isPartOf$fma260695'), ('Cytosol of epithelial cell','isPartOf$fma260691')]
    column_names_widget_middle = widgets.Dropdown(
        options = column_names_middle,
        style={'description_width': 'initial'},
        disabled = False,
        layout = Layout(width='400px', height='60px')
    )
    
    ## Set the font size using CSS
    font_size = '28px'  # Font size in CSS format

    # Generate the CSS code to update the font size
    css_code = f"""
    <style>
    .widget-dropdown select {{
        font-size: {font_size};
    }}
    .widget-dropdown .widget-label {{
        font-size: {font_size};
    }}
    </style>
    """

    
    def left_dropdown_eventhandler(change1):
        for i in selectedAnnotations[key]['q_'+element]:
            if 'entity$' in i:
                selectedAnnotations[key]['q_'+element].remove(i)
        selectedAnnotations[key]['q_'+element].append(change1.new)
        with open('./temporary files/selectedAnnotations.json', 'w') as f1:
            json.dump(selectedAnnotations, f1, indent=4, sort_keys=True, ensure_ascii=False)  
    def middle_dropdown_eventhandler(change2):
        for i in selectedAnnotations[key]['q_'+element]:
            if 'isPartOf$' in i:
                selectedAnnotations[key]['q_'+element].remove(i)
        selectedAnnotations[key]['q_'+element].append(change2.new)
        with open('./temporary files/selectedAnnotations.json', 'w') as f1:
            json.dump(selectedAnnotations, f1, indent=4, sort_keys=True, ensure_ascii=False) 

    column_names_widget_left.observe(left_dropdown_eventhandler, names='value')
    column_names_widget_middle.observe(middle_dropdown_eventhandler, names='value')
   
    # Display the widgets
    text_left=widgets.HTML(value="<h1><b><font color='salmon'>Physical entity:<b><h1>")
    display(text_left)
    display(column_names_widget_left)
    text_middle=widgets.HTML(value="<h1><b><font color='salmon'>Contained in:<b><h1>")
    display(text_middle)
    display(column_names_widget_middle)
    display(HTML(css_code))



    
    
    return selectedAnnotations

In [1]:
def calculate_v(pVals , *species):
    f = open("./temporary files/pVals.json")
    pVals = json.load(f)
    f = open("./temporary files/speciesStoi.json")
    speciesStoi = json.load(f)
    
    if singleSelection.value == 'Template 3':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        
        ao,ai = species
        Ao,Ai = ao**n_Ao, ai**n_Ai
        
        return (pVals[singleSelection.value]['p1']* Ao - pVals[singleSelection.value]['p2']*Ai)/ (pVals[singleSelection.value]['p3'] + pVals[singleSelection.value]['p4']*Ao + pVals[singleSelection.value]['p5']*Ai*Ao + pVals[singleSelection.value]['p6']*Ai)
    
    if singleSelection.value == 'Template 4':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        
        ao,ai,bo,bi = species
        Ao,Ai,Bo,Bi = ao**n_Ao, ai**n_Ai, bo**n_Bo, bi**n_Bi
        
        return (pVals[singleSelection.value]['p1']*Ao*Bi - pVals[singleSelection.value]['p2']*Ai*Bo)/ (pVals[singleSelection.value]['p3']*Ao + pVals[singleSelection.value]['p4']*Ai + pVals[singleSelection.value]['p5']*Bo + pVals[singleSelection.value]['p6']*Bi + pVals[singleSelection.value]['p7']*Ao*Bi + pVals[singleSelection.value]['p8']*Ai*Ao + pVals[singleSelection.value]['p9']*Ai*Bo + pVals[singleSelection.value]['p10']*Bo*Bi)
    
    if singleSelection.value == 'Template 7':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        n_Ci = speciesStoi[singleSelection.value]['q_Ci'][1]
        n_Co = speciesStoi[singleSelection.value]['q_Co'][1]
        
        ao,ai,bo,bi,co,ci = species
        Ao,Ai,Bo,Bi,Co,Ci = ao**n_Ao, ai**n_Ai, bo**n_Bo, bi**n_Bi, co**n_Co, ci**n_Ci
        
        return (pVals[singleSelection.value]['p1']*Ai*Bo*Co - pVals[singleSelection.value]['p2']*Ao*Bi*Ci)/(pVals[singleSelection.value]['p3']*Ai + pVals[singleSelection.value]['p4']*Ao + pVals[singleSelection.value]['p5']*Bi + pVals[singleSelection.value]['p6']*Bo + pVals[singleSelection.value]['p7']*Ci + pVals[singleSelection.value]['p8']*Co + pVals[singleSelection.value]['p9']*Ai*Ao + pVals[singleSelection.value]['p10']*Bi*Bo + pVals[singleSelection.value]['p11']*Ci*Co + pVals[singleSelection.value]['p12']*Ai*Bo*Co + pVals[singleSelection.value]['p13']*Bi*Ci*Ao )
    
    if singleSelection.value == 'Template 11':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        n_Ci = speciesStoi[singleSelection.value]['q_Ci'][1]
        n_Co = speciesStoi[singleSelection.value]['q_Co'][1]
        n_Di = speciesStoi[singleSelection.value]['q_Di'][1]
        n_Do = speciesStoi[singleSelection.value]['q_Do'][1]
        
        ao,ai,bo,bi,co,ci,do,di = species
        Ao,Ai,Bo,Bi,Co,Ci,Do,Di = ao**n_Ao, ai**n_Ai, bo**n_Bo, bi**n_Bi, co**n_Co, ci**n_Ci, do**n_Do, di**n_Di
        
        return (pVals[singleSelection.value]['p1']*Ai*Bo*Co*Do - pVals[singleSelection.value]['p2']*Ao*Bi*Ci*Di)/(pVals[singleSelection.value]['p3']*Ai + pVals[singleSelection.value]['p4']*Ao + pVals[singleSelection.value]['p5']*Bi + pVals[singleSelection.value]['p6']*Bo + pVals[singleSelection.value]['p7']*Ci + pVals[singleSelection.value]['p8']*Co + pVals[singleSelection.value]['p9']*Di + pVals[singleSelection.value]['p10']*Do + pVals[singleSelection.value]['p11']*Ai*Ao + pVals[singleSelection.value]['p12']*Bi*Bo + pVals[singleSelection.value]['p13']*Ci*Co + pVals[singleSelection.value]['p14']*Di*Do + pVals[singleSelection.value]['p15']*Ai*Bo*Co*Do + pVals[singleSelection.value]['p16']*Bi*Ci*Ao*Di )    
    
    if singleSelection.value == 'Template 5':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        
        ao,ai,bo,bi,V_m = species
        Ao, Ai, Bo, Bi = ao**n_Ao, ai**n_Ai, bo**n_Bo, bi**n_Bi
        F=96485
        R=8.31
        T=310
        return (pVals[singleSelection.value]['p1']*Bo*Ao*exp(F*V_m/(R*T))-pVals[singleSelection.value]['p2']*Ai*Bi)/(pVals[singleSelection.value]['p11']*Bi*Ai*Ao*exp(F*V_m/(R*T))+pVals[singleSelection.value]['p12']*Bi*Bo*Ai*Ao*exp(F*V_m/(R*T))+pVals[singleSelection.value]['p3']*Bi*Bo+pVals[singleSelection.value]['p4']*Bo*Ai+pVals[singleSelection.value]['p5']*Bo*Ao*exp(F*V_m/(R*T))+pVals[singleSelection.value]['p6']*Bi*Ai+pVals[singleSelection.value]['p7']*Bi*Ao*exp(F*V_m/(R*T))+pVals[singleSelection.value]['p8']*Bi*Bo*Ai+pVals[singleSelection.value]['p9']*Bi*Bo*Ao*exp(F*V_m/(R*T))+pVals[singleSelection.value]['p10']*Bo*Ai*Ao*exp(F*V_m/(R*T)))
    
    if singleSelection.value == 'Template 6':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        
        ao,ai,bo,bi,V_m = species
        Ao, Ai, Bo, Bi = ao**n_Ao, ai**n_Ai, bo**n_Bo, bi**n_Bi
        F=96485
        R=8.31
        T=310
        EXP = exp(F*V_m/(R*T))
        EXP2 = exp(n_Bo*F*V_m/(R*T))
        return (Bo*Bi*(pVals[singleSelection.value]['p1']*Ao*EXP2-pVals[singleSelection.value]['p2']*Ai))/(pVals[singleSelection.value]['p3']*Bi*Bo + pVals[singleSelection.value]['p4']*Bi*Bo*EXP + pVals[singleSelection.value]['p5']*Bi*Bo*EXP2+ pVals[singleSelection.value]['p6']*Bo*Ai + pVals[singleSelection.value]['p7']*Bo*Ai*EXP + pVals[singleSelection.value]['p8']*Bo*Ai*EXP2 + pVals[singleSelection.value]['p9']*Bo*Ao*EXP2+ pVals[singleSelection.value]['p10']*Bi*Ai + pVals[singleSelection.value]['p11']*Bi*Ao+ pVals[singleSelection.value]['p12']*Bi*Ao*EXP + pVals[singleSelection.value]['p13']*Bi*Bo*Ai + pVals[singleSelection.value]['p14']*Bi*Bo*Ai*EXP+ pVals[singleSelection.value]['p15']*Bi*Bo*Ai*EXP2 + pVals[singleSelection.value]['p16']*Bi*Bo*Ao*EXP2 + pVals[singleSelection.value]['p17']*Bi*Bo*Ao*EXP + pVals[singleSelection.value]['p18']*Bo*Ai*Ao*EXP2 + pVals[singleSelection.value]['p19']*Bi*Ai*Ao + pVals[singleSelection.value]['p20']*Bi*Bo*Ai*Ao*EXP2 + pVals[singleSelection.value]['p21']*Bi*Bo*Ai*Ao*EXP)
    
    if singleSelection.value == 'Template 8':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        ao,ai,bo,bi,V_m = species
        Ao, Ai, Bo, Bi = ao**n_Ao, ai**n_Ai, bo**n_Bo, bi**n_Bi
        F=96485
        R=8.31
        T=310
        EXP = exp(F*V_m/(R*T))
        EXP2 = exp(n_Ao*F*V_m/(R*T))
        return  (EXP2*(pVals[singleSelection.value]['p1']*Bo*Ao -pVals[singleSelection.value]['p2']*Bi*Ai))/(pVals[singleSelection.value]['p11']*Bi*Ai*Ao*EXP + pVals[singleSelection.value]['p3']*Bi*EXP + pVals[singleSelection.value]['p4']*Bi*EXP2+ pVals[singleSelection.value]['p5']*Bi*Ai*EXP2+ pVals[singleSelection.value]['p6']*Bo*Ai*EXP2+ pVals[singleSelection.value]['p7']*Bi*Ao*EXP+ pVals[singleSelection.value]['p8']*Ai*EXP+ pVals[singleSelection.value]['p9']*Ai*EXP2+ pVals[singleSelection.value]['p10']*Bi*Ao*EXP2)
    
    if singleSelection.value == 'Template 1':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        n_Ci = speciesStoi[singleSelection.value]['q_Ci'][1]
        n_Co = speciesStoi[singleSelection.value]['q_Co'][1]
        n_Di = speciesStoi[singleSelection.value]['q_Di'][1]
        n_Do = speciesStoi[singleSelection.value]['q_Do'][1]
        ao,ai,bo,bi,co,ci,do,di = species
        Ao,Ai,Bo,Bi,Co,Ci,Do,Di = ao**n_Ao, ai**n_Ai, bo**n_Bo, bi**n_Bi, co**n_Co, ci**n_Ci, do**n_Do, di**n_Di
        return Ai*Ao*Bi*Bo*(pVals[singleSelection.value]['p1']*Di*Co - pVals[singleSelection.value]['p2']*Do*Ci)/(pVals[singleSelection.value]['p11']*Ai*Ao*Bo*Di*Co+pVals[singleSelection.value]['p12']*Ai*Ao*Bo*Do*Ci+pVals[singleSelection.value]['p13']*Ai*Ao*Bo*Ci*Co+pVals[singleSelection.value]['p14']*Ai*Ao*Bi*Di*Co+pVals[singleSelection.value]['p15']*Ai*Ao*Bi*Do*Ci+pVals[singleSelection.value]['p16']*Ai*Ao*Bi*Ci*Co+pVals[singleSelection.value]['p17']*Ai*Ao*Bi*Bo*Do+pVals[singleSelection.value]['p18']*Ai*Ao*Bi*Bo*Co+pVals[singleSelection.value]['p19']*Ai*Ao*Bi*Bo*Di*Do+pVals[singleSelection.value]['p20']*Ai*Ao*Bi*Bo*Di*Co+pVals[singleSelection.value]['p3']*Ai*Ao*Bi*Bo*Di+pVals[singleSelection.value]['p21']*Ai*Ao*Bi*Bo*Do*Ci+pVals[singleSelection.value]['p22']*Ai*Ao*Bi*Bo*Ci*Co+pVals[singleSelection.value]['p4']*Ai*Ao*Bi*Bo*Ci+pVals[singleSelection.value]['p5']*Ao*Bi*Bo*Di*Co+pVals[singleSelection.value]['p6']*Ao*Bi*Bo*Do*Ci+pVals[singleSelection.value]['p7']*Ao*Bi*Bo*Ci*Co+pVals[singleSelection.value]['p8']*Ai*Bi*Bo*Di*Co+pVals[singleSelection.value]['p9']*Ai*Bi*Bo*Do*Ci+pVals[singleSelection.value]['p10']*Ai*Bi*Bo*Ci*Co)

    if singleSelection.value == 'Template 10':
        n_Ai = speciesStoi[singleSelection.value]['q_Ai'][1]
        n_Ao = speciesStoi[singleSelection.value]['q_Ao'][1]
        n_Bi = speciesStoi[singleSelection.value]['q_Bi'][1]
        n_Bo = speciesStoi[singleSelection.value]['q_Bo'][1]
        n_Ci = speciesStoi[singleSelection.value]['q_Ci'][1]
        n_Co = speciesStoi[singleSelection.value]['q_Co'][1]
        ao,ai,bo,bi,co,ci,V_m = species
        Ao,Ai,Bo,Bi,Co,Ci = ao**n_Ao, ai**n_Ai, bo**n_Bo, bi**n_Bi, co**n_Co, ci**n_Ci
        F=96485
        R=8.31
        T=310
        EXP = exp(F*V_m/(R*T))
        
        return (pVals[singleSelection.value]['p1']*Ao*Bo*Co*EXP - pVals[singleSelection.value]['p2']*Ai*Bi*Ci)/(pVals[singleSelection.value]['p11']*Ao*Co*EXP + pVals[singleSelection.value]['p12']*Ai*Bi + pVals[singleSelection.value]['p13']*Ao*Bo+pVals[singleSelection.value]['p14']*Bi*Ci+pVals[singleSelection.value]['p15']*Ai*Bi*Ci+pVals[singleSelection.value]['p16']*Ai*Bo*Co*EXP+pVals[singleSelection.value]['p17']*Ao*Bo*Co*EXP+pVals[singleSelection.value]['p18']*Ao*Bo*Ci+pVals[singleSelection.value]['p19']*Ai*Bi*Co*EXP+pVals[singleSelection.value]['p20']*Ao*Bi*Ci+pVals[singleSelection.value]['p3']*Ci+pVals[singleSelection.value]['p21']*Ai*Bi*Bo*Co*EXP+pVals[singleSelection.value]['p22']*Ai*Bi*Bo*Ci+pVals[singleSelection.value]['p23']*Ao*Bo*Ci*Co*EXP+pVals[singleSelection.value]['p24']*Ai*Bi*Ci*Co*EXP+pVals[singleSelection.value]['p25']*Ai*Ao*Bi*Ci+pVals[singleSelection.value]['p26']*Ai*Ao*Bo*Co*EXP+pVals[singleSelection.value]['p27']*Ao*Bi*Bo*Co*EXP+pVals[singleSelection.value]['p28']*Ao*Bi*Bo*Ci+pVals[singleSelection.value]['p29']*Ai*Bi*Bo*Ci*Co*EXP+pVals[singleSelection.value]['p30']*Ai*Ao*Bi*Bo*Co*EXP+pVals[singleSelection.value]['p4']*Co*EXP+pVals[singleSelection.value]['p31']*Ai*Ao*Bi*Bo*Ci+pVals[singleSelection.value]['p32']*Ao*Bi*Bo*Ci*Co*EXP+pVals[singleSelection.value]['p33']*Ai*Ao*Bi*Bo*Ci*Co*EXP+pVals[singleSelection.value]['p34']+pVals[singleSelection.value]['p5']*Ai+pVals[singleSelection.value]['p6']*Ao+pVals[singleSelection.value]['p7']*Bo*Co*EXP+pVals[singleSelection.value]['p8']*Ai*Ci+pVals[singleSelection.value]['p9']*Ao*Ci+pVals[singleSelection.value]['p10']*Ai*Co*EXP)
    

In [5]:
def update_figure(control_variable, control_value, stepTime, timespan, **const_values):

    time = np.linspace(0, timespan, 1000)
    f = open("./temporary files/pVals.json")
    pVals = json.load(f)
 

    V=[]
    if singleSelection.value == 'Template 3':
        if control_variable=='Ao':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ao']
                    else:
                        return control_value
                V.append(calculate_v(pVals, step_func(t), const_values['Ai']))

        if control_variable=='Ai':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ai']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], step_func(t)))
                                
    if singleSelection.value == 'Template 4':
        if control_variable=='Ao':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ao']
                    else:
                        return control_value
                V.append(calculate_v(pVals, step_func(t), const_values['Ai'], const_values['Bo'], const_values['Bi']))

        if control_variable=='Ai':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ai']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], step_func(t), const_values['Bo'], const_values['Bi']))
                
        if control_variable=='Bo':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Bo']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], step_func(t), const_values['Bi']))
                
        if control_variable=='Bi':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Bi']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], step_func(t)))
                
    if singleSelection.value == 'Template 7':
        if control_variable=='Ao':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ao']
                    else:
                        return control_value
                V.append(calculate_v(pVals, step_func(t), const_values['Ai'], const_values['Bo'], const_values['Bi'], const_values['Co'], const_values['Ci']))

        if control_variable=='Ai':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ai']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], step_func(t), const_values['Bo'], const_values['Bi'], const_values['Co'], const_values['Ci']))
                
        if control_variable=='Bo':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Bo']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], step_func(t), const_values['Bi'], const_values['Co'], const_values['Ci']))
                
        if control_variable=='Bi':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Bi']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], step_func(t), const_values['Co'], const_values['Ci']))
                
        if control_variable=='Co':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Co']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], const_values['Bi'], step_func(t), const_values['Ci']))
                
        if control_variable=='Ci':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ci']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], const_values['Bi'], const_values['Co'], step_func(t)))
                
    if singleSelection.value == 'Template 11' or singleSelection.value == 'Template 1':
        if control_variable=='Ao':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ao']
                    else:
                        return control_value
                V.append(calculate_v(pVals, step_func(t), const_values['Ai'], const_values['Bo'], const_values['Bi'], const_values['Co'], const_values['Ci'], const_values['Do'], const_values['Di']))

        if control_variable=='Ai':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ai']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], step_func(t), const_values['Bo'], const_values['Bi'], const_values['Co'], const_values['Ci'], const_values['Do'], const_values['Di']))
                
        if control_variable=='Bo':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Bo']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], step_func(t), const_values['Bi'], const_values['Co'], const_values['Ci'], const_values['Do'], const_values['Di']))
                
        if control_variable=='Bi':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Bi']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], step_func(t), const_values['Co'], const_values['Ci'], const_values['Do'], const_values['Di']))
                
        if control_variable=='Co':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Co']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], const_values['Bi'], step_func(t), const_values['Ci'], const_values['Do'], const_values['Di']))
                
        if control_variable=='Ci':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ci']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], const_values['Bi'], const_values['Co'], step_func(t), const_values['Do'], const_values['Di']))
                
        if control_variable=='Do':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Do']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], const_values['Bi'], const_values['Co'], const_values['Ci'], step_func(t), const_values['Di']))
                
        if control_variable=='Di':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Di']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], const_values['Bi'], const_values['Co'], const_values['Ci'], const_values['Do'], step_func(t)))
                
    if singleSelection.value == 'Template 5' or  singleSelection.value == 'Template 6'  or  singleSelection.value == 'Template 8':
        if control_variable=='Ao':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ao']
                    else:
                        return control_value
                V.append(calculate_v(pVals, step_func(t), const_values['Ai'], const_values['Bo'], const_values['Bi'], const_values['V_m']))

        if control_variable=='Ai':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ai']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], step_func(t), const_values['Bo'], const_values['Bi'], const_values['V_m']))
                
        if control_variable=='Bo':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Bo']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], step_func(t), const_values['Bi'], const_values['V_m']))
                
        if control_variable=='Bi':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Bi']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], step_func(t), const_values['V_m']))
                
        if control_variable=='V_m':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['V_m']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], const_values['Bi'], step_func(t)))
                

    if singleSelection.value == 'Template 10':
        if control_variable=='Ao':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ao']
                    else:
                        return control_value
                V.append(calculate_v(pVals, step_func(t), const_values['Ai'], const_values['Bo'], const_values['Bi'], const_values['Co'], const_values['Ci'], const_values['V_m']))

        if control_variable=='Ai':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ai']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], step_func(t), const_values['Bo'], const_values['Bi'], const_values['Co'], const_values['Ci'], const_values['V_m']))
                
        if control_variable=='Bo':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Bo']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], step_func(t), const_values['Bi'], const_values['Co'], const_values['Ci'], const_values['V_m']))
                
        if control_variable=='Bi':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Bi']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], step_func(t), const_values['Co'], const_values['Ci'], const_values['V_m']))
                
        if control_variable=='Co':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Co']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], const_values['Bi'], step_func(t), const_values['Ci'], const_values['V_m']))
                
        if control_variable=='Ci':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['Ci']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], const_values['Bi'], const_values['Co'], step_func(t), const_values['V_m']))
        
        if control_variable=='V_m':
            for t in time:
                def step_func(t):
                    if t < stepTime:
                        return const_values['V_m']
                    else:
                        return control_value
                V.append(calculate_v(pVals, const_values['Ao'], const_values['Ai'], const_values['Bo'], const_values['Bi'], const_values['Co'], const_values['Ci'], step_func(t)))

                


    # Update the plot
    matplotlib.rcParams.update({'font.size': 15})
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(time, V, color='m', linewidth=5, label='$V_{ss}$')
    ax.set_xlabel('Time (s)')
    ax.grid(color='gray', linestyle='-', linewidth=0.5)
    ax.set_ylabel('Flux ($u$mol/nL.s)')
    ax.legend(loc='best')
    formatted_control_value = f'{control_value:.2f}'
    formatted_control_variable = [f'$q_{control_variable}$' if control_variable != 'V_m' else f'${control_variable}$'][0]
    ax.set_title(f'Control variable: {formatted_control_variable}, Control value: {formatted_control_value}')
    plt.show()
